#Matrix Multiplication using numba 
We know that Matrix operations are very time consuming and Matrix multiplication is one of the prolonged process. To overcome this we try to use matrix multiplication. Here we’ll compare time required to perform Matrix multiplication with CPU and GPU. Also, we’ll try to find time required to compute matrix multiplication of two 10000x10000 matrices.

In [1]:
import time
import numba
import numpy as np
import numpy as np
from numba import cuda,jit

In [2]:
wA = 10000
hA = 10000
wB = hA
hB = wA

Firstly, we need to know some basics like what is a kernel, threads and blocks. \\
A **CUDA Kernel** is a function that is executed on GPU 'n' number of times  in parallel by 'n' different **CUDA threads**, as opposed to only once like regular functions. A group of threads is called a **CUDA block**. 
CUDA architecture limits the numbers of threads per block (1024 threads is the limit of per block).

Here, we use just in time compiler (jit) with CUDA. `cuda.jit` helps in a  low-level entry point to the CUDA features in Numba. 

In this function, first we define kernel matrix that has parameters as the input matrices that are copied to work on device memory. We set the number of threads as `cuda.gridsize(1)`; it returns the number of threads that are present in the grid block. We then set the current thread using `cuda.grid(1)`.  Then we carry out the matrix multiplication.

The second function takes normal matrices and copies them to device using `cuda.to_device(X)`. Once it is copied then it calls the kernel and passes them to the above function to get the result of matrix multiplication.
A note must be taken of blocks per grid means number of blocks and threads per block is same as number of threads.

In [3]:
@cuda.jit
def matmul_kernel(A, B, C):
    num_of_threads = cuda.gridsize(1)
    curr_id = cuda.grid(1)
    rows_num = C.shape[0]
    cols_num = C.shape[1]
    idx_range = A.shape[1]
    for mid in range(curr_id, rows_num*cols_num, num_of_threads):
        row = mid // cols_num
        col = mid - (row*cols_num)
        my_sum = 0.0
        for idx in range(0, idx_range):
            my_sum += A[row, idx] * B[idx, col]
        C[row, col] = my_sum

def matmul_gpu(X, Y):
    # Allocate the output matrix in GPU memory using cuda.to_device
    #
    # invoke the dot kernel with 1 threadBlock with 1024 threads
    #
    # copy the output matrix from GPU to cpu using copy_to_host()
    gpu_mat1 = cuda.to_device(X)
    gpu_mat2 = cuda.to_device(Y)
    res = np.zeros(shape=(X.shape[0], Y.shape[1]), dtype=np.float32)
    gpu_mult_res = cuda.to_device(res)
    threads_per_block = 1024
    blocks_per_grid = 512
    matmul_kernel[blocks_per_grid, threads_per_block](
        gpu_mat1, gpu_mat2, gpu_mult_res)
    mult_res = gpu_mult_res.copy_to_host()
    return mult_res

In [4]:
a = np.random.rand(wA,hA)
b = np.random.rand(wB,hB)

In [5]:
start = time.perf_counter()
gpumatrix1 = matmul_gpu(a,b)
end = time.perf_counter()

dt_gpu1 = end-start

In [6]:
start = time.perf_counter()
gpumatrix2 = matmul_gpu(a,b)
end = time.perf_counter()

dt_gpu2 = end-start

In [7]:
start = time.perf_counter()
gpumatrix3 = matmul_gpu(a,b)
end = time.perf_counter()

dt_gpu3 = end-start

In [8]:
start = time.perf_counter()
cpumatrix = np.dot(a,b)
end=time.perf_counter()

#print results
dt_cpu1 = end-start

In [9]:
start = time.perf_counter()
cpumatrix2 = np.matmul(a,b)
end=time.perf_counter()

#print results
dt_cpu2 = end-start

In [10]:
print("Matrix multipliation first time on GPU = %f s" % dt_gpu1)
print("Matrix multipliation second time on GPU = %f s" % dt_gpu2)
print("Matrix multipliation third time on GPU = %f s" % dt_gpu3)
print("Matrix multipliation on CPU using dot(.) operator in = %f s" % dt_cpu1)
print("Matrix multipliation on CPU using matmul function in = %f s" % dt_cpu2)

Matrix multipliation first time on GPU = 44.338945 s
Matrix multipliation second time on GPU = 43.917687 s
Matrix multipliation third time on GPU = 43.834267 s
Matrix multipliation on CPU using dot(.) operator in = 58.588512 s
Matrix multipliation on CPU using matmul function in = 58.583081 s


We can clearly see the time difference of using GPU and CPU. CPU is 1.5 times slower than GPU in this case. Also, we check the results generated by GPU and CPU are same (rounded upto 4 decimal places). This result indicates that GPU is not only faster but also accurate.

In [11]:
np.allclose(np.around(cpumatrix, decimals=4),np.around(gpumatrix1, decimals=4))

True

In [12]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Sep  7 17:04:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    57W / 149W |    441MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------